In [2]:
import pandas as pd
import numpy as np

In [3]:
# data = pd.read_excel("C:\\Users\\win10\\Desktop\\wind0716.xlsx",
#                     parse_dates=[4,5])

data = pd.read_excel('C:\\Users\\win10\\Desktop\\0806担保.xlsx')
data.head(10)

,xpstkcd,xpname,bdbstkcd,bdbjc,qs,zz,dbxpgx
0,7,全新好,48,康达尔,2001,2002,公司本身
1,7,全新好,48,康达尔,2001,2004,公司本身
2,7,全新好,48,康达尔,2002,2003,公司本身
3,7,全新好,48,康达尔,2003,2003,公司本身
4,7,全新好,48,康达尔,2003,2004,公司本身
5,9,中国宝安,13,*ST石化A,2000,2001,公司本身
6,9,中国宝安,13,*ST石化A,2001,2002,公司本身
7,9,中国宝安,13,*ST石化A,2003,2004,公司本身
8,9,中国宝安,34,神州数码,2000,2003,公司本身
9,9,中国宝安,34,神州数码,2000,2001,公司本身


In [12]:
def time_complete(start,end,sep):
    from pandas.tseries.offsets import Day
    from pandas._libs.tslib import NaTType
    for i in range(len(start)):
        if isinstance(start.iat[i],NaTType):
            try:
                start.iat[i] = end.iat[i]-Day(np.ceil(sep.iat[i]*365))
            except ValueError:
                pass
        elif isinstance(end.iat[i],NaTType):
            try:
                end.iat[i] = start.iat[i]+Day(np.ceil(sep.iat[i]*365))
            except ValueError:
                pass
    return pd.concat([start,end],axis=1)
qs_zz = time_complete(data.qs,data.zz,data.dbqx)
# data.combine_first(qs_zz)
data = pd.concat([data.drop(data[['qs','zz']],axis=1),qs_zz],axis=1)
data.head(10)

,xpstkcd,xpname,bdbstkcd,bdbjc,dbqx,riqi,year,bdbname,dbname,dbxpgx,qs,zz
0,7,全新好,48,康达尔,2.7,11/28/2001,12/31/2003,深圳市康达尔（集团）股份有限公司,深圳市赛格达声股份有限公司,公司本身,2001-11-28,2004-07-29
1,7,全新好,48,康达尔,0.5,12/30/2002,6/30/2004,深圳市康达尔（集团）股份有限公司,深圳市赛格达声股份有限公司,公司本身,2003-01-02,2003-07-01
2,7,全新好,48,康达尔,0.9,5/29/2003,12/31/2005,深圳市康达尔（集团）股份有限公司,深圳市赛格达声股份有限公司,公司本身,2003-05-29,2004-04-28
3,7,全新好,48,康达尔,1.0,7/19/2002,12/31/2004,深圳市康达尔（集团）股份有限公司,深圳市赛格达声股份有限公司,公司本身,2002-07-19,2003-07-19
4,7,全新好,48,康达尔,0.9,12/20/2002,6/30/2004,深圳市康达尔（集团）股份有限公司,深圳市赛格达声股份有限公司,公司本身,2002-12-27,2003-11-26
5,7,全新好,48,康达尔,1.0,5/29/2003,6/30/2004,深圳市康达尔（集团）股份有限公司,深圳市赛格达声股份有限公司,公司本身,2003-05-29,2004-05-28
6,7,全新好,48,康达尔,0.6,12/29/2003,6/30/2004,深圳市康达尔（集团）股份有限公司,深圳市赛格达声股份有限公司,公司本身,2003-12-29,2004-07-29
7,7,全新好,48,康达尔,1.0,11/28/2001,6/30/2004,深圳市康达尔（集团）股份有限公司,深圳市赛格达声股份有限公司,公司本身,2001-11-28,2002-11-28
8,7,全新好,48,康达尔,0.6,12/29/2003,12/31/2004,深圳市康达尔（集团）股份有限公司,深圳市赛格达声股份有限公司,公司本身,2003-12-29,2004-08-04
9,7,全新好,48,康达尔,1.0,7/19/2002,6/30/2004,深圳市康达尔（集团）股份有限公司,深圳市赛格达声股份有限公司,公司本身,2002-07-17,2003-07-16


In [5]:
def gen_year(df):
    data = pd.DataFrame()
    for g_num,g_data in df.groupby(df.index):
        try:
            year = pd.period_range(start=g_data.qs.values[0],end=g_data.zz.values[0],freq="A-DEC")
        except ValueError:
            year=[]
        year = pd.Series(year,name='expand_year',
                        index=list(range(g_data.index[0],g_data.index[0]+len(year))))
        g_data = pd.concat([g_data,year],axis=1,join='outer')
#         g_data=g_data.fillna(method='ffill').dropna()
        g_data=g_data.fillna(method='ffill')
        data = pd.concat([data,g_data],ignore_index=True)
    return data.drop_duplicates()
expand_data = gen_year(data)

In [26]:
final_data = expand_data.drop(['qs','zz'],axis=1).drop_duplicates()
# final_data['expand_year'].value_counts()
# expand_data.head(20)

2004    319
2005    269
2003    262
2006    204
2007    137
2008    127
2009    111
2010    103
2002     98
2011     77
2012     73
2013     68
2014     62
2015     55
2016     51
2001     44
2017     36
2000     25
2018     24
1999     14
1998     10
2019      8
1997      6
2020      4
1996      4
2021      2
1995      2
2023      1
1992      1
1993      1
2024      1
2025      1
2026      1
2022      1
2027      1
1994      1
Freq: A-DEC, Name: expand_year, dtype: int64

In [27]:
# expand_data.to_excel("C:\\Users\\win10\\Desktop\\hc.xlsx",index=False)
final_data.to_excel("C:\\Users\\win10\\Desktop\\hc.xlsx",index=False)